In [2]:
import torch
from torch.optim import Adam
from tqdm import tqdm

import scipy
from scipy.io import loadmat
from scipy.io import savemat
import numpy as np


def frobenius_norm(tensor):
    return torch.norm(tensor, 'fro')


def loss_function(X, Y, C):
    return frobenius_norm(X - C @ Y @ C.inverse())


def cayley_transform(A):
    I = torch.eye(A.size(0), dtype=A.dtype, device=A.device)
    return torch.linalg.solve(I + A, I - A)

def cayley_numpy(A):
    I =np.eye((Aa-Aa.T).shape[0])
    return np.linalg.solve(I+A,I-A)

learning_rate = 0.01
max_iterations = 1500



# Now do DSA with A~ high components models with ncomps=116 (80% variance in PODs)

In [3]:
DataFold='/Users/user/PycharmProjects/PacManMain/data/DMDmat/UPDATED/AllComponents/'
Xall=loadmat(DataFold+'/X_Amatrix_fullAllComps.mat')['XA']
Yall=loadmat(DataFold+'/Y_Amatrix_fullAllComps.mat')['YA']


X=torch.from_numpy(np.float32(Xall))
Y=torch.from_numpy(np.float32(Yall))


max_iterations=3000

Run DSA

In [4]:
n = X.size(0)

# Initialize a skew-symmetric matrix A

A = torch.randn((n, n), requires_grad=True)
A.data = A.data - A.data.T  # Making A skew-symmetric but preserving leaf status
# Use ADAM optimizer
optimizer = Adam([A], lr=learning_rate)

for iteration in range(max_iterations):
    optimizer.zero_grad()

    # Ensure C is orthogonal via Cayley transform of skew-symmetric A
    C = cayley_transform(A)
    loss = loss_function(X, Y, C)

    # Compute gradients
    loss.backward()

    # Update A using gradients
    optimizer.step()

    if iteration % 100 == 0:
        print(f'Iteration {iteration}, Loss: {loss.item()}')

# Final optimized C
C_optimized = cayley_transform(A)
C_optimized=C_optimized.detach().numpy()
Optimized_loss=loss.detach().numpy()

#compute angular
num = np.trace(X.detach().numpy().T @ C_optimized @ Y.detach().numpy() @ np.linalg.inv(C_optimized))


denom = torch.norm(X,p = 'fro')*torch.norm(Y,p = 'fro')
denom=denom.detach().numpy()
angoptimalscore = np.cos(np.arccos(num/denom))


/Users/user/anaconda3/envs/main/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Iteration 0, Loss: 3.543290376663208
Iteration 100, Loss: 2.5587775707244873
Iteration 200, Loss: 2.1216747760772705
Iteration 300, Loss: 1.893959403038025
Iteration 400, Loss: 1.7469725608825684
Iteration 500, Loss: 1.638789415359497
Iteration 600, Loss: 1.5567067861557007
Iteration 700, Loss: 1.4933186769485474
Iteration 800, Loss: 1.4388370513916016
Iteration 900, Loss: 1.392980694770813
Iteration 1000, Loss: 1.3530372381210327
Iteration 1100, Loss: 1.3185499906539917
Iteration 1200, Loss: 1.2924251556396484
Iteration 1300, Loss: 1.265633225440979
Iteration 1400, Loss: 1.240638256072998
Iteration 1500, Loss: 1.2199738025665283
Iteration 1600, Loss: 1.198548436164856
Iteration 1700, Loss: 1.177392601966858
Iteration 1800, Loss: 1.1614265441894531
Iteration 1900, Loss: 1.1435043811798096
Iteration 2000, Loss: 1.128136157989502
Iteration 2100, Loss: 1.1101489067077637
Iteration 2200, Loss: 1.0958455801010132
Iteration 2300, Loss: 1.0833172798156738
Iteration 2400, Loss: 1.0711603164672

50:50 split comparison within system now

## Perm with random x 50

In [5]:

DataFold='/Users/user/PycharmProjects/PacManMain/data/DMDmat/UPDATED/AllComponents/'

C_on_perm=np.empty([116,116,50])
loss_on_perm=np.zeros((50))
angpermscore=np.zeros((50))
for perm in range(50):
    Xall=loadmat(DataFold+'/X_Amatrix_fullAllComps.mat')['XA']
    Yall=loadmat(DataFold+'/Y_Amatrix_fullAllComps.mat')['YA']

    X=torch.from_numpy(np.float32(Xall))
    n = X.size(0)

    #apply rotation using random skew symmetric orthogonal transform
    Aa=np.random.randn(n,n)
    Yall=cayley_numpy(Aa-Aa.T)*Yall * scipy.linalg.inv(cayley_numpy(Aa-Aa.T))

    Y=torch.from_numpy(np.float32(Yall))

    # Initialize a skew-symmetric matrix A

    A = torch.randn((n, n), requires_grad=True)
    A.data = A.data - A.data.T  # Making A skew-symmetric but preserving leaf status
    # Use ADAM optimizer
    optimizer = Adam([A], lr=learning_rate)

    for iteration in range(max_iterations):
        optimizer.zero_grad()

        # Ensure C is orthogonal via Cayley transform of skew-symmetric A
        C = cayley_transform(A)
        loss = loss_function(X, Y, C)

        # Compute gradients
        loss.backward()

        # Update A using gradients
        optimizer.step()

        if iteration % 100 == 0:
            print(f'Iteration {iteration}, Loss: {loss.item()}')

    # Final optimized C
    C_perm = cayley_transform(A)
    C_perm=C_perm.detach().numpy()
    C_on_perm[:,:,perm]=C_perm
    losstmp = loss.detach().numpy()
    loss_on_perm[perm]=losstmp

    #compute angular
    num = np.trace(X.detach().numpy().T @ C_perm @ Y.detach().numpy() @ np.linalg.inv(C_perm))


    denom = torch.norm(X,p = 'fro')*torch.norm(Y,p = 'fro')
    denom=denom.detach().numpy()
    angpermscore[perm] = np.cos(np.arccos(num/denom))


Iteration 0, Loss: 3.156742811203003
Iteration 100, Loss: 2.9119744300842285
Iteration 200, Loss: 2.8325586318969727
Iteration 300, Loss: 2.7709357738494873
Iteration 400, Loss: 2.7277019023895264
Iteration 500, Loss: 2.700035572052002
Iteration 600, Loss: 2.6897881031036377
Iteration 700, Loss: 2.6406564712524414
Iteration 800, Loss: 2.62481427192688
Iteration 900, Loss: 2.6052916049957275
Iteration 1000, Loss: 2.594083786010742
Iteration 1100, Loss: 2.574978828430176
Iteration 1200, Loss: 2.5598013401031494
Iteration 1300, Loss: 2.553746223449707
Iteration 1400, Loss: 2.540450096130371
Iteration 1500, Loss: 2.530935764312744
Iteration 1600, Loss: 2.521026611328125
Iteration 1700, Loss: 2.510446310043335
Iteration 1800, Loss: 2.5034849643707275
Iteration 1900, Loss: 2.4963228702545166
Iteration 2000, Loss: 2.4852254390716553
Iteration 2100, Loss: 2.476865291595459
Iteration 2200, Loss: 2.473109722137451
Iteration 2300, Loss: 2.4665677547454834
Iteration 2400, Loss: 2.457742214202881
I

/var/folders/tv/_yg34tj15s306l4m7k_wf0qw0000gq/T/ipykernel_44290/3063630798.py:55: RuntimeWarning: invalid value encountered in arccos
  angpermscore[perm] = np.cos(np.arccos(num/denom))


Iteration 0, Loss: 3.1366727352142334
Iteration 100, Loss: 2.9348220825195312
Iteration 200, Loss: 2.8553154468536377
Iteration 300, Loss: 2.8125646114349365
Iteration 400, Loss: 2.7720718383789062
Iteration 500, Loss: 2.7519423961639404
Iteration 600, Loss: 2.7258260250091553
Iteration 700, Loss: 2.6893670558929443
Iteration 800, Loss: 2.677729606628418
Iteration 900, Loss: 2.645343065261841
Iteration 1000, Loss: 2.6419050693511963
Iteration 1100, Loss: 2.6219730377197266
Iteration 1200, Loss: 2.607372760772705
Iteration 1300, Loss: 2.587630033493042
Iteration 1400, Loss: 2.628061294555664
Iteration 1500, Loss: 2.571596384048462
Iteration 1600, Loss: 2.5628223419189453
Iteration 1700, Loss: 2.5498476028442383
Iteration 1800, Loss: 2.5393435955047607
Iteration 1900, Loss: 2.5388803482055664
Iteration 2000, Loss: 2.521383762359619
Iteration 2100, Loss: 2.539430618286133
Iteration 2200, Loss: 2.5137085914611816
Iteration 2300, Loss: 2.5104329586029053
Iteration 2400, Loss: 2.519509792327

In [6]:
mdic = {'optimizedC':[C_optimized],'optoloss':[Optimized_loss],'angularoptimal': [angoptimalscore],'loss_on_perm_random': [loss_on_perm],'C_on_perm' : [C_on_perm],'angperscore': [angpermscore]}
savemat(DataFold+'/C_optimized_full_dmd_ACC.mat',mdic)



## Perm with eigen maintained

# Now do DSA in PMD with A~ high components models with ncomps=197 (80% variance in PODs)

In [7]:
DataFold='/Users/user/PycharmProjects/PacManMain/data/DMDmat/UPDATED/AllComponents/'
Xall=loadmat(DataFold+'/X_Amatrix_fullAllComps_PMD.mat')['XA']
Yall=loadmat(DataFold+'/Y_Amatrix_fullAllComps_PMD.mat')['YA']


X=torch.from_numpy(np.float32(Xall))
Y=torch.from_numpy(np.float32(Yall))


max_iterations=3000

In [8]:
n = X.size(0)

# Initialize a skew-symmetric matrix A

A = torch.randn((n, n), requires_grad=True)
A.data = A.data - A.data.T  # Making A skew-symmetric but preserving leaf status
# Use ADAM optimizer
optimizer = Adam([A], lr=learning_rate)

for iteration in range(max_iterations):
    optimizer.zero_grad()

    # Ensure C is orthogonal via Cayley transform of skew-symmetric A
    C = cayley_transform(A)
    loss = loss_function(X, Y, C)

    # Compute gradients
    loss.backward()

    # Update A using gradients
    optimizer.step()

    if iteration % 100 == 0:
        print(f'Iteration {iteration}, Loss: {loss.item()}')

# Final optimized C
C_optimized = cayley_transform(A)
C_optimized=C_optimized.detach().numpy()
Optimized_loss=loss.detach().numpy()

#compute angular
num = np.trace(X.detach().numpy().T @ C_optimized @ Y.detach().numpy() @ np.linalg.inv(C_optimized))


denom = torch.norm(X,p = 'fro')*torch.norm(Y,p = 'fro')
denom=denom.detach().numpy()
angoptimalscore = np.cos(np.arccos(num/denom))

Iteration 0, Loss: 5.882142543792725
Iteration 100, Loss: 4.629274845123291
Iteration 200, Loss: 4.01328182220459
Iteration 300, Loss: 3.6558966636657715
Iteration 400, Loss: 3.4177539348602295
Iteration 500, Loss: 3.2521886825561523
Iteration 600, Loss: 3.120637893676758
Iteration 700, Loss: 3.002737283706665
Iteration 800, Loss: 2.9100944995880127
Iteration 900, Loss: 2.823728322982788
Iteration 1000, Loss: 2.74371075630188
Iteration 1100, Loss: 2.6726346015930176
Iteration 1200, Loss: 2.610757827758789
Iteration 1300, Loss: 2.5532736778259277
Iteration 1400, Loss: 2.5018062591552734
Iteration 1500, Loss: 2.469520092010498
Iteration 1600, Loss: 2.423938751220703
Iteration 1700, Loss: 2.3738059997558594
Iteration 1800, Loss: 2.338578701019287
Iteration 1900, Loss: 2.301952838897705
Iteration 2000, Loss: 2.274930715560913
Iteration 2100, Loss: 2.2530088424682617
Iteration 2200, Loss: 2.2187283039093018
Iteration 2300, Loss: 2.189997434616089
Iteration 2400, Loss: 2.178333282470703
Iter

Do perm DSA with PMD

In [9]:
DataFold='/Users/user/PycharmProjects/PacManMain/data/DMDmat/UPDATED/AllComponents/'

C_on_perm=np.empty([197,197,50])
loss_on_perm=np.zeros((50))
angpermscore=np.zeros((50))
for perm in range(50):
    Xall=loadmat(DataFold+'/X_Amatrix_fullAllComps_PMD.mat')['XA']
    Yall=loadmat(DataFold+'/Y_Amatrix_fullAllComps_PMD.mat')['YA']

    X=torch.from_numpy(np.float32(Xall))
    n = X.size(0)

    #apply rotation using random skew symmetric orthogonal transform
    Aa=np.random.randn(n,n)
    Yall=cayley_numpy(Aa-Aa.T)*Yall * scipy.linalg.inv(cayley_numpy(Aa-Aa.T))

    Y=torch.from_numpy(np.float32(Yall))

    # Initialize a skew-symmetric matrix A

    A = torch.randn((n, n), requires_grad=True)
    A.data = A.data - A.data.T  # Making A skew-symmetric but preserving leaf status
    # Use ADAM optimizer
    optimizer = Adam([A], lr=learning_rate)

    for iteration in range(max_iterations):
        optimizer.zero_grad()

        # Ensure C is orthogonal via Cayley transform of skew-symmetric A
        C = cayley_transform(A)
        loss = loss_function(X, Y, C)

        # Compute gradients
        loss.backward()

        # Update A using gradients
        optimizer.step()

        if iteration % 100 == 0:
            print(f'Iteration {iteration}, Loss: {loss.item()}')

    # Final optimized C
    C_perm = cayley_transform(A)
    C_perm=C_perm.detach().numpy()
    C_on_perm[:,:,perm]=C_perm
    losstmp = loss.detach().numpy()
    loss_on_perm[perm]=losstmp
    #compute angular
    num = np.trace(X.detach().numpy().T @ C_perm @ Y.detach().numpy() @ np.linalg.inv(C_perm))


    denom = torch.norm(X,p = 'fro')*torch.norm(Y,p = 'fro')
    denom=denom.detach().numpy()
    angpermscore[perm] = np.cos(np.arccos(num/denom))

Iteration 0, Loss: 4.674692630767822
Iteration 100, Loss: 4.486717700958252
Iteration 200, Loss: 4.379359245300293
Iteration 300, Loss: 4.318807601928711
Iteration 400, Loss: 4.261453628540039
Iteration 500, Loss: 4.211609363555908
Iteration 600, Loss: 4.183804512023926
Iteration 700, Loss: 4.151187896728516
Iteration 800, Loss: 4.097973823547363
Iteration 900, Loss: 4.079065322875977
Iteration 1000, Loss: 4.064431190490723
Iteration 1100, Loss: 4.008897304534912
Iteration 1200, Loss: 3.9841392040252686
Iteration 1300, Loss: 3.9730417728424072
Iteration 1400, Loss: 3.956218957901001
Iteration 1500, Loss: 3.9541056156158447
Iteration 1600, Loss: 3.9185853004455566
Iteration 1700, Loss: 3.901047468185425
Iteration 1800, Loss: 3.875500440597534
Iteration 1900, Loss: 3.853907585144043
Iteration 2000, Loss: 3.8470351696014404
Iteration 2100, Loss: 3.823117256164551
Iteration 2200, Loss: 3.815187692642212
Iteration 2300, Loss: 3.802658796310425
Iteration 2400, Loss: 3.790266990661621
Iterati

/var/folders/tv/_yg34tj15s306l4m7k_wf0qw0000gq/T/ipykernel_44290/3053493548.py:54: RuntimeWarning: invalid value encountered in arccos
  angpermscore[perm] = np.cos(np.arccos(num/denom))


Iteration 0, Loss: 4.639807224273682
Iteration 100, Loss: 4.458405017852783
Iteration 200, Loss: 4.421462059020996
Iteration 300, Loss: 4.355259895324707
Iteration 400, Loss: 4.329426288604736
Iteration 500, Loss: 4.251631736755371
Iteration 600, Loss: 4.180776119232178
Iteration 700, Loss: 4.139563083648682
Iteration 800, Loss: 4.082249164581299
Iteration 900, Loss: 4.051419734954834
Iteration 1000, Loss: 4.027443885803223
Iteration 1100, Loss: 3.9881343841552734
Iteration 1200, Loss: 3.954993486404419
Iteration 1300, Loss: 3.941052198410034
Iteration 1400, Loss: 3.8958261013031006
Iteration 1500, Loss: 3.8871712684631348
Iteration 1600, Loss: 3.8637006282806396
Iteration 1700, Loss: 3.846083402633667
Iteration 1800, Loss: 3.823432445526123
Iteration 1900, Loss: 3.812854766845703
Iteration 2000, Loss: 3.7943272590637207
Iteration 2100, Loss: 3.779947280883789
Iteration 2200, Loss: 3.7551589012145996
Iteration 2300, Loss: 3.7390339374542236
Iteration 2400, Loss: 3.7344796657562256
Iter

In [10]:
mdic = {'optimizedC':[C_optimized],'optoloss':[Optimized_loss],'angularoptimal': [angoptimalscore],'loss_on_perm_random': [loss_on_perm],'C_on_perm' : [C_on_perm],'angperscore': [angpermscore]}
savemat(DataFold+'/C_optimized_full_dmd_PMD.mat',mdic)
